In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

ruta = "modelo/data/output/resultados.xlsx"

output_dir = "graficos_resultados"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Cargando datos desde: {ruta}")
print(f"Guardando gráficos en: {output_dir}/")

df_gen = pd.read_excel(ruta, sheet_name="Generadores")
df_scc = pd.read_excel(ruta, sheet_name="Cortocircuito")
df_freq = pd.read_excel(ruta, sheet_name="Frecuencia")

print("Datos cargados exitosamente.")

# --- Gráfico 1: Despacho de Generación (Stacked Bar) ---
print("Generando Gráfico 1: Despacho de Generación...")
df_gen['generator'] = 'Gen ' + df_gen['generator'].astype(str)
gen_pivot = df_gen.pivot_table(index='time', columns='generator', values='p', aggfunc='sum')
gen_pivot = gen_pivot.fillna(0)
    
plt.figure(figsize=(14, 7))
ax = gen_pivot.plot(kind='bar', stacked=True, figsize=(14, 7), width=0.8)
ax.set_title('Despacho de Generación por Hora', fontsize=16)
ax.set_xlabel('Hora (time)', fontsize=12)
ax.set_ylabel('Potencia (p) [MW]', fontsize=12)
ax.legend(title='Generador', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xticks(rotation=45)
ax.grid(axis='y', linestyle='--', alpha=0.7)
    
if gen_pivot.index.min() == 1:
    ax.set_xticks(range(0, 24))
    ax.set_xticklabels(range(1, 25))
else:
    ax.set_xticklabels(gen_pivot.index)
    plt.tight_layout(rect=[0, 0, 0.88, 1])
plot_path_gen = os.path.join(output_dir, "despacho_generacion.png")
plt.savefig(plot_path_gen)
plt.close()
print(f"Gráfico 1 guardado en: {plot_path_gen}")

# --- Gráfico 2: Validación de la Restricción SCC (Corregido) ---
print("Generando Gráfico 2: Validación de la Restricción SCC...")

try:
    # --- INICIO DE LA MODIFICACIÓN ---
    # El gráfico original era engañoso. Ahora comparamos
    # la 'I_total' (SCC real) con la 'I_limit_real' (límite real).

    # 1. Verificar que la columna 'I_limit_real' exista
    if 'I_limit_real' not in df_scc.columns:
        print("Error: La columna 'I_limit_real' no se encontró en 'resultados.xlsx - Cortocircuito'.")
        print("Por favor, asegúrate de estar corriendo la última versión de 'main.jl' (la que te pasé) que exporta esta columna.")
        raise Exception("Columna 'I_limit_real' faltante.")

    # 2. Para cada hora, encontramos el bus que tiene la SCC MÍNIMA.
    #    idxmin() nos da el índice de la fila (bus) con el peor 'I_total'
    idx_worst_bus_per_hour = df_scc.groupby('time')['I_total'].idxmin()
    df_worst_case = df_scc.loc[idx_worst_bus_per_hour]

    # 3. Ahora df_worst_case tiene la fila completa (I_total, I_limit_real)
    #    del bus más débil en cada hora.

    plt.figure(figsize=(14, 7))
    
    # 4. Graficar la SCC mínima real (Línea Azul)
    ax = df_worst_case.plot(x='time', y='I_total', marker='o', label='SCC Mínima del Sistema ($I_{total}$)', color='dodgerblue', linewidth=2, figsize=(14,7))
    
    # 5. Graficar el LÍMITE REAL en ese bus (Línea Roja)
    #    Usamos 'ax=ax' para que grafique en el mismo eje
    df_worst_case.plot(x='time', y='I_limit_real', marker='x', linestyle='--', label='Límite Real Requerido ($I_{limit\_real}$)', color='red', linewidth=2, ax=ax)
    
    ax.set_title('Validación de Restricción SCC: Peor Bus del Sistema por Hora', fontsize=16)
    ax.set_xlabel('Hora (time)', fontsize=12)
    ax.set_ylabel('SCC [p.u.]', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 6. (Opcional) Rellenar el área donde se cumple la restricción
    ax.fill_between(
        df_worst_case['time'], 
        df_worst_case['I_total'], 
        df_worst_case['I_limit_real'], 
        where=(df_worst_case['I_total'] >= df_worst_case['I_limit_real']), 
        color='green', 
        alpha=0.2, 
        label='Margen de Cumplimiento'
    )
    
    times = df_worst_case['time']
    ax.set_xticks(ticks=times[::1])
    ax.set_xticklabels([str(t) for t in times[::1]])
    ax.set_xlim(times.min(), times.max())
    
    plt.tight_layout()
    
    plot_path_scc = os.path.join(output_dir, "scc_minima_sistema.png")
    plt.savefig(plot_path_scc)
    plt.close()
    print(f"Gráfico 2 guardado en: {plot_path_scc}")

except Exception as e:
    print(f"Error al generar Gráfico 2: {e}")

# --- Gráfico 3: Inercia Equivalente del Sistema ---
# (Este gráfico está bien, no se necesita cambiar)
print("Generando Gráfico 3: Inercia del Sistema...")
plt.figure(figsize=(14, 7))
ax = df_freq.plot(x='time', y='inertia_equiv', kind='line', marker='s', label='Inercia Equivalente', color='green', linewidth=2, figsize=(14, 7))
ax.set_title('Inercia Equivalente del Sistema por Hora', fontsize=16)
ax.set_xlabel('Hora (time)', fontsize=12)
ax.set_ylabel('Inercia (inertia_equiv) [MWs]', fontsize=12)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)
times = df_freq['time']
ax.set_xticks(ticks=times[::1])
ax.set_xticklabels([str(t) for t in times[::1]])
ax.set_xlim(times.min(), times.max())
plt.tight_layout()
plot_path_freq = os.path.join(output_dir, "inercia_sistema.png")
plt.savefig(plot_path_freq)
plt.close()
print(f"Gráfico 3 guardado en: {plot_path_freq}")

# --- Gráfico 4: RoCoF del Sistema ---
print("Generando Gráfico 4: RoCoF del Sistema...")
rocof_limit = 0.5
plt.figure(figsize=(14, 7))
plt.plot(df_freq['time'], df_freq['rocof'], marker='x', linestyle='-', label='RoCoF del Sistema', color='purple', linewidth=2)
plt.axhline(y=rocof_limit, color='red', linestyle='--', linewidth=2, label=f'Límite RoCoF = {rocof_limit} Hz/s')
plt.title('RoCoF (Rate of Change of Frequency) del Sistema por Hora', fontsize=16)
plt.xlabel('Hora (time)', fontsize=12)
plt.ylabel('RoCoF [Hz/s]', fontsize=12)
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
times = df_freq['time']
plt.xticks(ticks=times[::1], labels=[str(t) for t in times[::1]])
plt.xlim(times.min(), times.max())
max_val = max(df_freq['rocof'].max(), rocof_limit)
plt.ylim(0, max_val * 1.1)
plt.tight_layout()
plot_path_rocof = os.path.join(output_dir, "rocof_sistema.png")
plt.savefig(plot_path_rocof)
plt.close()
print(f"Gráfico 4 guardado en: {plot_path_rocof}")

print("\n--- Proceso de graficación completado ---")

Cargando datos desde: modelo/data/output/resultados.xlsx
Guardando gráficos en: graficos_resultados/
Datos cargados exitosamente.
Generando Gráfico 1: Despacho de Generación...
Gráfico 1 guardado en: graficos_resultados\despacho_generacion.png
Generando Gráfico 2: Validación de la Restricción SCC...
Gráfico 2 guardado en: graficos_resultados\scc_minima_sistema.png
Generando Gráfico 3: Inercia del Sistema...
Gráfico 3 guardado en: graficos_resultados\inercia_sistema.png
Generando Gráfico 4: RoCoF del Sistema...
Gráfico 4 guardado en: graficos_resultados\rocof_sistema.png

--- Proceso de graficación completado ---


<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

Mapeo de Buses

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import os
import numpy as np
import matplotlib.lines as mlines

ruta_output = "modelo/data/output/resultados.xlsx"

ruta_input = "modelo/data/input/datos_finales30.xlsx" 

output_dir = "graficos_resultados"
output_filename = os.path.join(output_dir, "scc_mapa_sld_final.png")

edges = [
    (1, 2), (1, 3), (2, 4), (3, 4), (2, 5), (2, 6), (4, 6), (5, 7), 
    (6, 7), (6, 8), (6, 9), (6, 10), (9, 11), (9, 10), (4, 12), (12, 13), 
    (12, 14), (12, 15), (12, 16), (14, 15), (16, 17), (15, 18), (18, 19), 
    (19, 20), (10, 20), (10, 17), (10, 21), (10, 22), (21, 22), (15, 23), 
    (22, 24), (23, 24), (24, 25), (25, 26), (25, 27), (28, 27), (27, 29), 
    (27, 30), (29, 30), (8, 28), (6, 28)
]

bus_positions = {
    1: (0.1, 0.9), 2: (0.3, 0.9), 3: (0.1, 0.7), 4: (0.3, 0.7),
    5: (0.5, 0.9), 6: (0.5, 0.7), 7: (0.7, 0.9), 8: (0.7, 0.6),
    9: (0.5, 0.5), 10: (0.5, 0.3), 11: (0.6, 0.5), 12: (0.2, 0.5),
    13: (0.05, 0.5), 14: (0.05, 0.35), 15: (0.2, 0.3), 16: (0.2, 0.6),
    17: (0.4, 0.4), 18: (0.2, 0.15), 19: (0.3, 0.1), 20: (0.4, 0.1),
    21: (0.6, 0.2), 22: (0.6, 0.1), 23: (0.3, 0.0), 24: (0.45, 0.0),
    25: (0.6, 0.0), 26: (0.7, 0.0), 27: (0.8, 0.2), 28: (0.9, 0.6),
    29: (0.8, 0.1), 30: (0.9, 0.0)
}

print(f"Cargando datos de resultados desde: {ruta_output}")
print(f"Cargando datos de buses desde: {ruta_input}")

# --- 1. Cargar y Procesar Datos ---
try:
    df_scc = pd.read_excel(ruta_output, sheet_name="Cortocircuito")
    df_bus_data = pd.read_excel(ruta_input, sheet_name="BUS DATA")
except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar un archivo.")
    print(f"Detalle: {e}")
    print("Por favor, asegúrate de que las rutas 'ruta_output' y 'ruta_input' sean correctas.")
    exit()
except Exception as e:
    print(f"Error al leer los archivos Excel: {e}")
    exit()

print("Procesando peor margen de SCC y tipos de bus...")

if 'I_limit_real' not in df_scc.columns or 'I_total' not in df_scc.columns:
    print("Error: Columnas 'I_total' o 'I_limit_real' no encontradas.")
    exit()

df_scc['scc_margin'] = df_scc['I_total'] - df_scc['I_limit_real']
worst_margin_per_bus = df_scc.groupby('bus_id')['scc_margin'].min()
scc_min_abs_per_bus = df_scc.groupby('bus_id')['I_total'].min()
bus_type_map = pd.Series(df_bus_data.Tipo.values, index=df_bus_data.id).to_dict()

try:
    demand_cols = [col for col in df_bus_data.columns if str(col).startswith('t') and str(col)[1:].isdigit()]
    if not demand_cols:
        df_bus_data['has_demand'] = False
    else:
        df_bus_data['has_demand'] = (df_bus_data[demand_cols].sum(axis=1) > 0)
    bus_has_demand = pd.Series(df_bus_data.has_demand.values, index=df_bus_data.id).to_dict()
except Exception as e:
    bus_has_demand = {}

# --- 2. Crear el Gráfico (Grafo) ---
G = nx.Graph()
G.add_nodes_from(scc_min_abs_per_bus.index)
G.add_edges_from(edges)

# --- 3. Definir Colormap ---
cmap = plt.cm.coolwarm_r 
v_min = worst_margin_per_bus.min()
v_max = worst_margin_per_bus.max()
norm_for_cbar = plt.Normalize(vmin=v_min, vmax=v_max)

# --- 4. Dibujar el Diagrama ---
print(f"Generando diagrama en: {output_filename}")
fig, ax = plt.subplots(figsize=(20, 14))

# 4.1. Dibujar líneas (ramas)
nx.draw_networkx_edges(G, bus_positions, ax=ax, alpha=0.5, edge_color='gray')

# 4.2. Definir tamaños de íconos/offsets
bus_bar_length = 0.015  # Largo de la barra de bus
symbol_offset = 0.05    # Distancia vertical del símbolo a la barra
arrow_length = 0.05
arrow_head_width = 0.015

# 4.3. Iterar y dibujar cada bus manualmente
for bus_id in G.nodes():
    if bus_id not in bus_positions: continue
    
    x, y = bus_positions[bus_id]
    bus_type = bus_type_map.get(bus_id, 0)
    margin = worst_margin_per_bus.get(bus_id, 0)
    color = cmap(norm_for_cbar(margin)) # Color basado en el margen

    # A. Dibujar la "Barra" del bus
    ax.plot([x - bus_bar_length, x + bus_bar_length], [y, y], 
            color=color, 
            linewidth=8, 
            solid_capstyle='butt')

    # B. Dibujar Símbolo de Generador Síncrono (SG)
    if bus_type in [1, 2]:
        gen_y = y + symbol_offset
        # --- MODIFICACIÓN: ÍCONO MÁS GRANDE ---
        ax.plot(x, gen_y, marker='o', markersize=20, 
                markerfacecolor='none', markeredgecolor='black', markeredgewidth=1.5)
        ax.text(x, gen_y, '~', fontsize=22, fontweight='bold', 
                ha='center', va='center')
        ax.plot([x, x], [y, gen_y - 0.013], color='black', linewidth=1) # Ajustar conexión

    # C. Dibujar Símbolo de Generador IBG (Eólico)
    if bus_type == 3:
        gen_y = y + symbol_offset
        # --- MODIFICACIÓN: ÍCONO MÁS GRANDE ---
        ax.plot(x, gen_y, marker='^', markersize=20, 
                markerfacecolor='none', markeredgecolor='black', markeredgewidth=1.5)
        ax.plot([x, x], [y, gen_y - 0.015], color='black', linewidth=1) # Ajustar conexión

    # D. Dibujar Símbolo de Carga (Load)
    if bus_has_demand.get(bus_id, False):
        load_y_start = y - 0.005 
        load_y_end = y - arrow_length
        # --- MODIFICACIÓN: ÍCONO MÁS GRANDE ---
        ax.arrow(x, load_y_start, 0, (load_y_end - load_y_start), 
                 head_width=arrow_head_width, head_length=0.02, 
                 fc='black', ec='black', length_includes_head=True)

    # E. Dibujar Etiquetas de Texto
    scc_val = scc_min_abs_per_bus.get(bus_id, 0)
    
    # --- MODIFICACIÓN: POSICIÓN DE TEXTO ---
    # ID del Bus (A la derecha de la barra)
    ax.text(x + 0.02, y, str(bus_id), 
            ha='left', va='center', fontsize=12, fontweight='bold')
    
    # Valor de SCC (Debajo de la barra, más abajo para no chocar)
    ax.text(x, y - (arrow_length + 0.02), f"{scc_val:.2f} p.u.", 
            ha='center', va='top', fontsize=10)

# --- 5. Añadir Título, Barra de Color y Leyenda de Íconos ---
ax.set_title('Mapa de Margen de SCC (Peor Caso en 24h)', fontsize=20)
ax.set_xlim(-0.05, 1.05) 
ax.set_ylim(-0.1, 1.05)
ax.axis('off')

# Barra de Color
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm_for_cbar)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, shrink=0.8, aspect=30, pad=0.01)
cbar.set_label('Margen de SCC (I_total - I_limit_real) - Rojo = Peor Margen', fontsize=14, weight='bold')

# Leyenda para los íconos
bus_bar = mlines.Line2D([], [], color='gray', marker='s', linestyle='None', 
                         markersize=10, markerfacecolor='gray', label='Barra de Bus (Color por Margen SCC)')
load_arrow = mlines.Line2D([], [], color='black', marker=r'$\downarrow$', linestyle='None', 
                           markersize=15, label='Carga (Demanda)')
sg_icon = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=12, 
                        markerfacecolor='none', markeredgewidth=1.5, label='Gen. Síncrono')
ibg_icon = mlines.Line2D([], [], color='black', marker='^', linestyle='None', markersize=12, 
                         markerfacecolor='none', markeredgewidth=1.5, label='Gen. IBG (Eólico)')

# --- MODIFICACIÓN: POSICIÓN DE LEYENDA ---
ax.legend(handles=[bus_bar, load_arrow, sg_icon, ibg_icon], 
          loc='lower left', fontsize=12, title="Simbología")

# --- 6. Guardar y Finalizar ---
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
plt.savefig(output_filename, bbox_inches='tight')
plt.close()

print("--- Proceso de graficación completado ---")

Cargando datos de resultados desde: modelo/data/output/resultados.xlsx
Cargando datos de buses desde: modelo/data/input/datos_finales30.xlsx
Procesando peor margen de SCC y tipos de bus...
Generando diagrama en: graficos_resultados\scc_mapa_sld_final.png
--- Proceso de graficación completado ---
